In [2]:
import optuna
import numpy as np
import pandas as pd

from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler

from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping

import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# warnings.simplefilter(action='ignore', category=UserWarning)

In [3]:
# Função para calcular o KS
def ks_metric(y_true, y_pred_proba):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
    ks = max(tpr - fpr)
    return ks

In [4]:
data = pd.read_csv('./data/creditcard.csv')
data.columns = data.columns.str.lower()
data.head()

,time,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v21,v22,v23,v24,v25,v26,v27,v28,amount,class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
# Separando as transações legítimas (fraud = 0) e fraudulentas (fraud = 1)
X = data.drop(['time', 'class'], axis=1).copy()
y = data['class'].copy()

# Dividindo em treino e teste
X_train_main, X_test_main, y_train_main, y_test_main = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = RobustScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train_main), columns=X_train_main.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test_main), columns=X_test_main.columns)

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_scaled, y_train_main, test_size=0.25, random_state=42)

X_train = X_train.reset_index().drop('index', axis=1)
X_valid = X_valid.reset_index().drop('index', axis=1)
X_test = X_test_scaled.reset_index().drop('index', axis=1)

y_train = pd.DataFrame(y_train).reset_index().drop('index', axis=1)['class']
y_valid = pd.DataFrame(y_valid).reset_index().drop('index', axis=1)['class']
y_test = pd.DataFrame(y_test_main).reset_index().drop('index', axis=1)['class']

In [7]:
# Função de objetivo do Optuna

def objective(trial):
    # Seleção de features (0 ou 1 para incluir/excluir a feature)
    selected_features = [i for i in range(X_train.shape[1]) if trial.suggest_categorical(f"feature_{i}", [0, 1]) == 1]
    
    if len(selected_features) == 0:  # Se não há features selecionadas, pular o trial
        return 0

    # Subconjunto das features selecionadas
    X_train_selected = X_train.iloc[:, selected_features]
    X_valid_selected = X_valid.iloc[:, selected_features]


    # Hiperparâmetros a serem otimizados
    encoding_dim = trial.suggest_int('encoding_dim', 2, 16, step=2)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'sigmoid'])
    optimizer_name = trial.suggest_categorical('optimizer', ['adam', 'rmsprop', 'sgd'])
    loss_function = trial.suggest_categorical('loss', ['binary_crossentropy', 'mean_squared_error', 'mean_absolute_error'])
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    epochs = trial.suggest_int('epochs', 10, 100, step=10)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)

    # Ajuste do otimizador
    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'rmsprop':
        optimizer = RMSprop(learning_rate=learning_rate)
    else:
        optimizer = SGD(learning_rate=learning_rate)

    # Construindo o Autoencoder com os hiperparâmetros do trial
    input_dim = X_train_selected.shape[1]  # Número de features
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(encoding_dim, activation=activation)(input_layer)
    decoded = Dense(input_dim, activation='sigmoid')(encoded)

    autoencoder = Model(inputs=input_layer, outputs=decoded)
    autoencoder.compile(optimizer=optimizer, loss=loss_function)

    # Configurando EarlyStopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=3)

    # Treinando o Autoencoder
    history = autoencoder.fit(X_train_selected[y_train == 0], X_train_selected[y_train == 0], 
                              epochs=epochs, 
                              batch_size=batch_size, 
                              shuffle=True, 
                              validation_data=(X_valid_selected[y_valid == 0], X_valid_selected[y_valid == 0]), 
                              callbacks=[early_stopping], 
                              verbose=0)

    # Calculando o erro de reconstrução
    X_train_pred = autoencoder.predict(X_train_selected, verbose=0)
    mse_train = np.mean(np.power(X_train_selected - X_train_pred, 2), axis=1)

    X_valid_pred = autoencoder.predict(X_valid_selected, verbose=0)
    mse_valid = np.mean(np.power(X_valid_selected - X_valid_pred, 2), axis=1)

    # Ajustar a escala de saída
    scaler = MinMaxScaler() 
    y_train_pred_proba = scaler.fit_transform(mse_train.values.reshape(-1, 1)).ravel()
    y_valid_pred_proba = scaler.transform(mse_valid.values.reshape(-1, 1)).ravel()

    # Calculando o KS para o teste
    ks_valid = ks_metric(y_valid, y_valid_pred_proba)

    return ks_valid  # Optuna tentará maximizar essa métrica

# Rodar o Optuna
n_jobs = max(1, os.cpu_count() - 2)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=500, n_jobs=n_jobs)

# Resultado do melhor trial
print(f'Melhor KS: {study.best_value}')
print(f'Melhores Hiperparâmetros: {study.best_params}')

[I 2024-09-16 12:10:18,299] A new study created in memory with name: no-name-03167e97-4cf2-4cba-9a4c-b7406e806fc0
[I 2024-09-16 12:13:31,613] Trial 4 finished with value: 0.7801396093534481 and parameters: {'feature_0': 1, 'feature_1': 1, 'feature_2': 0, 'feature_3': 1, 'feature_4': 1, 'feature_5': 0, 'feature_6': 0, 'feature_7': 1, 'feature_8': 0, 'feature_9': 1, 'feature_10': 1, 'feature_11': 0, 'feature_12': 0, 'feature_13': 0, 'feature_14': 1, 'feature_15': 0, 'feature_16': 1, 'feature_17': 1, 'feature_18': 1, 'feature_19': 0, 'feature_20': 0, 'feature_21': 0, 'feature_22': 1, 'feature_23': 1, 'feature_24': 1, 'feature_25': 0, 'feature_26': 1, 'feature_27': 1, 'feature_28': 1, 'encoding_dim': 6, 'activation': 'tanh', 'optimizer': 'rmsprop', 'loss': 'mean_absolute_error', 'batch_size': 128, 'epochs': 20, 'learning_rate': 0.02276453184945834}. Best is trial 4 with value: 0.7801396093534481.
[I 2024-09-16 12:17:41,919] Trial 6 finished with value: 0.7867522569547744 and parameters: {'

Melhor KS: 0.8469422252032476
Melhores Hiperparâmetros: {'feature_0': 0, 'feature_1': 1, 'feature_2': 0, 'feature_3': 1, 'feature_4': 1, 'feature_5': 1, 'feature_6': 1, 'feature_7': 0, 'feature_8': 0, 'feature_9': 1, 'feature_10': 1, 'feature_11': 1, 'feature_12': 0, 'feature_13': 1, 'feature_14': 1, 'feature_15': 0, 'feature_16': 1, 'feature_17': 0, 'feature_18': 0, 'feature_19': 0, 'feature_20': 0, 'feature_21': 0, 'feature_22': 0, 'feature_23': 0, 'feature_24': 0, 'feature_25': 0, 'feature_26': 0, 'feature_27': 0, 'feature_28': 1, 'encoding_dim': 2, 'activation': 'sigmoid', 'optimizer': 'rmsprop', 'loss': 'mean_squared_error', 'batch_size': 32, 'epochs': 70, 'learning_rate': 0.02295420468278658}


100 testes, 1 job, 2h40min

Melhor KS: 0.8461736795890512

Melhores Hiperparâmetros: {'feature_0': 0, 'feature_1': 1, 'feature_2': 0, 'feature_3': 0, 'feature_4': 1, 'feature_5': 1, 'feature_6': 0, 'feature_7': 0, 'feature_8': 0, 'feature_9': 0, 'feature_10': 1, 'feature_11': 0, 'feature_12': 0, 'feature_13': 1, 'feature_14': 1, 'feature_15': 1, 'feature_16': 1, 'feature_17': 0, 'feature_18': 1, 'feature_19': 1, 'feature_20': 0, 'feature_21': 0, 'feature_22': 0, 'feature_23': 1, 'feature_24': 1, 'feature_25': 0, 'feature_26': 0, 'feature_27': 0, 'feature_28': 1, 'encoding_dim': 10, 'activation': 'sigmoid', 'optimizer': 'sgd', 'loss': 'binary_crossentropy', 'batch_size': 64, 'epochs': 20, 'learning_rate': 4.2423550697632125e-05}